# Template Translation Detection of Egyptian Articles

## Supervised Classification Algorithms:

### Best-performing Classifier, XGBoost:

In [1]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

df = pd.read_csv("../../Experimental-Setups/arzwiki-20240101-all-20k.csv")
df

,page_title,total_edits,total_editors,total_bytes,total_chars,total_words,page_text,page_text_cleaned,label
0,كريستينا نوير,27,6,1685,460,65,كريستينا نوير christina noir يوليه ممثلة بورنو...,كريستينا نوير يوليه ممثلة بورنوجرافية امريكية ...,human-generated
1,ارنست هيفليجر,2,2,1060,381,67,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,ارنست هيفليجر كان معلم موسيقى مغنى اوبرا مغنى ...,template-translated
2,ناديه مصطفى,19,8,1841,637,113,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,نادية مصطفى اكتوبر مغنية مصرية بدأت مشوارها ال...,human-generated
3,جامعات عمان,10,6,1810,767,113,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,الجامعات ومؤسسات التعليم العالي فى سلطنه عمان ...,human-generated
4,الحرب السكندريه,51,16,12077,6192,1083,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,يوليوس قيصر خاض حرب اسكندريه الحرب السكندريه ح...,human-generated
...,...,...,...,...,...,...,...,...,...
19995,عبد الخالق صالح,12,5,1070,467,81,عبدالخالق صالح abdelkhalek saleh ممثل مصرى من ...,عبدالخالق صالح ممثل مصرى من مواليد تخرج فى كلي...,human-generated
19996,چيم رونى,4,2,916,301,53,چيم رونى لاعب كورة قدم من المملكه المتحده حيات...,چيم رونى لاعب كورة قدم من المملكه المتحده حيات...,template-translated
19997,البطوله الوطنيه الاولى 1963,2,2,872,315,51,البطوله الوطنيه الاولى országos bajnokság هوا ...,البطوله الوطنيه الاولى هوا موسم رياضى فى كورة ...,template-translated
19998,دايان لين,13,7,1157,350,59,دايان لين بسنة دايان لين هيا ممثله امريكيه اتو...,دايان لين بسنة دايان لين هيا ممثله امريكيه اتو...,human-generated


#### Load Input Features:

In [2]:
import json
import numpy as np
import pandas as pd

def fetch_embeddings(page_text, dataframe_name, embeddings_file):
    idx = np.where(dataframe_name["page_text_cleaned"] == page_text)[0][0]
    page_title = dataframe_name['page_title'][idx]
    return embeddings_file[page_title]
    
embeddings_files = ["../../Experimental-Setups/arzwiki-20240101-all-20k-camelbert-embeddings.json"]

for file in embeddings_files:
    with open(file) as json_file:
        fetched_page_text_embeddings = json.load(json_file)
        df[f'{file.split("/")[-1].split("-")[-2]}_page_text_embeddings'] = df['page_text_cleaned'].apply(lambda page_text: fetch_embeddings(page_text, df, fetched_page_text_embeddings))

label2id = {'human-generated': 0, 'template-translated': 1}

df['label'] = df['label'].map(label2id)

In [3]:
import numpy as np
import pandas as pd

def prepare_features(df, features, model=None):
    X = []
    for i in range(df.shape[0]):
        x = []
        x.append(df['total_edits'][i])
        x.append(df['total_editors'][i])
        x.append(df['total_bytes'][i])
        x.append(df['total_chars'][i])
        x.append(df['total_words'][i])
        X.append(np.hstack([x, df['camelbert_page_text_embeddings'].to_list()[i]])) # Both page_metadata + page_text_embeddings
    y = df['label'].to_list()
    return X, y

### Save Best-performing Classifier, XGBoost:

In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

model, features = "camelbert", "metadata+embeddings"

X, y = prepare_features(df, features=features, model=model) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, stratify=y, shuffle=True, random_state=2024)

xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)
xgb_classifier.save_model(f"XGBoost_metadata+camelbert_embeddings.model")

### Test Best-performing Classifier, XGBoost:

In [5]:
from xgboost import XGBClassifier

loaded_model = XGBClassifier()

loaded_model.load_model(f"XGBoost_metadata+camelbert_embeddings.model")

id2label = {0:'human-generated', 1:'template-translated'}

id2label[int(loaded_model.predict(np.array([X_test[0]])))], y_test[0] # Test with one example, X_test[0], and its truth label y_test[0]

('template-translated', 1)